In this worksheet we will learn about the "simulation box", a very useful trick that allows us to study systems without imposing hard boundaries (like the walls of a container).

The idea behind the simulation box is that all of the $N$ atoms are placed in a cubic box of side length $L$. If this was a real box then the walls of the box would effect the system we are studying. However, the walls of our box have the special property that if an atom hits the wall on, say, the right side, it simply re-enters the simulation box from the left side. In a way the right side and the left side of the box are glued together, and similarly the top and bottom sides and the front and back sides of the box. This treatment of the box "walls" is referred to as Periodic Boundary Conditions (PBCs).

Let's see how we can implement this in our code. As always we start by importing the scientific and plotting functions so that we can use them in this notebook:

In [ ]:
%pylab inline

First we reuse from the last worksheet the function definitions for the pair potential and the resulting force:

In [ ]:
def u(r):
    if r >= rc:
        return 0.0
    else:
        return 4.0 * (r**-12 - r**-6) - 4*(rc**-12 - rc**-6)

def f(r):
    if r >= rc:
        return 0.0
    else:
        return -24.*(2*r**-13 - r**-7)

The argument `r` of these functions is the distance between two atoms, say, atoms $i$ and $j$. Previously we computed this distance simply as $r = |\boldsymbol{r}_j - \boldsymbol{r}_i|$. However, when we use periodic boundary conditions, an atom close to the left side of the box might be very close to another atom on the right side of the box. Furthermore, since there is now an inifnite number of periodic copies of atom $j$, we have to decide which of those contribute to the interaction with atom $i$. Here we appply the Minimum Image Convention: out of all periodic copies of atom $j$, only the one closest to atom $i$ contributes to the $i-j$ interaction.

This might sound complicated, but actually requires adding only a single line to the code that calculates the total potential energy (and the same line is added to the code that calculates forces): if the size of the box is stored in a three-component array $(L_x,L_y,L_z)$, then we can calculate the potential energy with the following function:

In [ ]:
def compute_potential_energy():
    """Computes the total potential energy of the system."""

    N = positions.shape[0]                      # there are as many atoms as there are positions
    
    Epot = 0.
    for i in range(0, N-1):
        for j in range(i+1, N):
            rij = positions[j] - positions[i]   # rij is the raw distance vector from particle i to particle j
            rij -= box * np.round(rij / box)    # apply periodic boundary condition / minimum image convention 
            r = sqrt(dot(rij, rij))             # calculate the magnitude of the distance: r = |rij| = sqrt( r_ij * r_ij )

            Epot += u(r)                        # add potential energy of this pair to total

    # return the copmuted energy to the calling program
    return Epot

The same line is added to the force calculation routine:

In [ ]:
def compute_forces():
    """Computes forces between all pairs of particles."""

    N = positions.shape[0]                      # there are as many atoms as there are positions

    # define an array where we will store the force acting on each particle
    forces = np.zeros ((N,3), dtype=float)

    # loop over all particle pairs (i,j), and calculate the force those particle exert on each other
    for i in range(0, N-1):
        for j in range(i+1, N):
            rij = positions[j] - positions[i]   # rij is the distance vector from particle i to particle j
            rij -= box * np.round(rij / box)    # apply periodic boundary condition / minimum image convention 
            r = sqrt(np.dot (rij, rij))         # calculate the magnitude of the distance: r = |rij| = sqrt( r_ij * r_ij )

            forces[i] += f(r) * rij / r         # add force to particle i
            forces[j] -= f(r) * rij / r         # add force to particle j

    # return the computed force array to the calling program
    return forces

Nothing changes in our code to compute the kinetic energy or to perform a Velocity Verlet timestep:

In [ ]:
def compute_kinetic_energy():
    """Computes the total kinetic energy of the system."""

    Ekin = 0.5 * np.sum(velocities**2)
    return Ekin


In [ ]:
def perform_timestep():
    global positions, velocities, forces # we have to tell python that we are modifying global objects
    velocities += 0.5 * dt * forces      # Step 1: calcuate velocities at time t + dt/2    
    positions += dt * velocities         # Step 2: calculate positions at time t + dt
    forces = compute_forces()            # Step 3: calculate forces for new positions at time t + dt
    velocities += 0.5 * dt * forces      # Step 4: calculate velocities at time t + dt
    return positions, velocities, forces

We would again like to save configurations so that we can look at them in VMD. This time we will use a slightly different file format (called a GRO file) which allows us to also store the size of the simulation box. While the coordinates of the atoms throughout our simulation may not be within our simulation box, we will use for each atom the one periodic image that lies within the box for output.

In [ ]:
def write_configuration(f):
    positions_in_box = positions - box * floor(positions / box)   # map all atom positions into the simulation box
    N = positions.shape[0]
    print (file=f)
    print (N, file=f)
    for i, pos in enumerate(positions_in_box):
        print ("%5d%-5s%5s%5d%8.3f%8.3f%8.3f" % (i, "Ar", "Ar", i, 0.376*pos[0], 0.376*pos[1], 0.376*pos[2]), file=f)
    print (0.376*box[0], 0.376*box[1], 0.376*box[2], file=f)

That's it! Notice that we added only two lines of code (in fact, we added the same line twice).

## Collision of two (almost) hard spheres

Let's revisit the collision of two particles interacting through the Weeks-Chandler-Andersen potential, which is very similar to the hard sphere potential:

In [ ]:
rc = 2**(1./6.)

Let's specify the timestep:

In [ ]:
dt = 0.01

We repeat our example from the last worksheet, but this time we have a periodically replicated simulation box:

In [ ]:
# set the initial condition: positions and velocities of all atoms, as well as the simulation box size:

box = array((10.,10.,10.))
positions = array([[0,5,5],[5,5,5]], dtype=float)
velocities = array([[1,0,0],[0,0,0]], dtype=float)

# now we reserve some space to store the x coordinates of the two atoms, as well as the potential end kinetic energy of the system:

Nsteps = 2500

x1 = zeros (Nsteps)
x2 = zeros (Nsteps)
Epot = zeros (Nsteps)
Ekin = zeros (Nsteps)

grofilename = "output.gro"
grofile = open(grofilename,"w")

forces = compute_forces()
for step in range(Nsteps):
    write_configuration(grofile)
    x1[step] = positions[0,0]
    x2[step] = positions[1,0]
    Epot[step] = compute_potential_energy()
    Ekin[step] = compute_kinetic_energy()

    perform_timestep()

grofile.close()

Done! Let's plot the x-coordinates of our two atoms as a function of timestep:

In [ ]:
plot(x1,label="Atom 1")
plot(x2,label="Atom 2")
xlabel("Timestep")
ylabel("x coordinate")
legend(loc="lower right")

Here is a plot of the potential, kinetic, and total energy as a function of timestep:

In [ ]:
plot(Epot,label="Potential Energy")
plot(Ekin,label="Kinetic Energy")
plot(Epot+Ekin,label="Total Energy")
xlabel("Timestep")
legend(loc="lower right")

#### Exercise 1: Explain what is going on in these two plots! Watch the trajectory in VMD. Remember to change the "Drawing Method" to "VDW" (under Graphics -> Representatons). While you are there, you might also want to change the "Coloring Method" to "ResID" so you can distinguish the two atoms.

## A condensed system of Lennard-Jones particles

We now consider a system of 125 atoms that interact through the pairwise additive Lnnard-Jones potential, truncated and shifted at $r_c = 2.5 \sigma$:

In [ ]:
rc = 2.5

This time we load the initial condition from a file. This file contains a 126 rows of 3 numbers: the first row is the side length of the simulation box in the x, y, and z direction, and the remaining 125 rows are the (x,y,z) coordinates of the atoms. We set the initial velocities of the atoms to zero:

In [ ]:
configuration = loadtxt("N125_L7.469.dat")
box = configuration[0]
positions = configuration[1:]
velocities = zeros_like(positions)

Let's run the Molecular Dynamics code! We will start with a relatively short simulation of 500 timesteps, but even that will take a little while (perhaps a couple of minutes, depending on your computer). 

In [ ]:
Nsteps = 500

grofilename = "output.gro"
grofile = open(grofilename,"w")          # open file to save trajectories

energyfilename = "energy.dat"
energyfile = open(energyfilename,"w")    # open file to save energies

forces = compute_forces()                # compute force for initial condition
for step in range(Nsteps):
    write_configuration(grofile)         # save configuration to file
    print (step*dt, compute_potential_energy(), compute_kinetic_energy(), file=energyfile)    # save energies to file
    perform_timestep()                   # perform Velocity Verlet timestep

energyfile.close()
grofile.close()

Once this is done we can look at the potential and kinetic energy throughout the simulation:

In [ ]:
energy = loadtxt(energyfilename)
times = energy[:,0]    # the first column
Epot = energy[:,1]     # the second column
Ekin = energy[:,2]     # the third column
plot (times, Epot, label="Potential Energy")
plot (times, Ekin, label="Kinetic Energy")
plot (times, Epot+Ekin, label="Total Energy")
xlabel("Time")
legend(loc="lower right")

#### Exercise 2: Watch the simulation in VMD! It might be helpful to visualize the extent of the simulation box, which you can do by typing `pbc box` into the VMD command window.

This is working just fine, but it is rather slow. The reason is that the code calculates all forces and energies at every timestep, both of which require a loop over all pairs of particles. For our system of 125 atoms there are 125*124/2 = 7750 unique pairs, and the system loops over all of those (twice) in every timestep. There are several algorithms that reduce this number, but they are beyond the scope of this class. Nevertheless, we can rewrite the code in a slightly faster way by eliminating the two nested `for` loops in the force and energy calculation.

First, let's look at how long the force calculation of our system takes:

In [ ]:
%timeit compute_forces()

On my laptop a single call to the `compute_forces` function takes about 200 milliseconds. Let's rewrite this in the following way:

In [ ]:
fvec = vectorize(f)

def compute_forces_new():
    """Computes forces between all pairs of particles."""

    rij = positions[newaxis,:,:]-positions[:,newaxis,:]          # rij is the NxNxd matrix where rij[i,j] is now positions[j]-positions[i]
    rij -= box * np.round(rij / box)                             # apply minimum image convention
    r = sqrt(sum(rij*rij,axis=-1))                               # r is now the NxN matrix of distances
    fill_diagonal(r,1.)                                          # set diagonal elements of the distance matrix to some arbitrary non-zero value to avoid dividing by zero when evaluating the force
    ff = fvec(r)/r
    np.fill_diagonal(ff,0.)                                      # zero out the diagonal elements of the force matrix
    forces = np.sum(ff[:,:,newaxis] * rij, axis=1)               # forces is an Nxd array where forces[i] is the force acting on particle i

    return forces

In comparison with our previous implementation above we here omitted the two nested for loops, and used instead operations that work on all pairs simultaneously. You do not need to worry about the details of how this works, as there is no new physics to be learned here. However, let's check how long this code takes to run:

In [ ]:
%timeit compute_forces_new()

On my laptop this takes only about 8 milliseconds, corresponding to a 25x speedup. Nice! Let's verify that the result from the new code gives the same result (within numerical accuracy) as the old one:

In [ ]:
allclose (compute_forces(), compute_forces_new())

Great! Let's use the new, faster `compute_forces` routine from now on:

In [ ]:
compute_forces = compute_forces_new

Similarly we can speed up the calculation of the potential energy:

In [ ]:
uvec = vectorize(u)

def compute_potential_energy():
    """Computes the total potential energy of the system."""

    rij = positions[newaxis,:,:]-positions[:,newaxis,:]          # rij is the NxNxd matrix where rij[i,j] is now positions[j]-positions[i]
    rij -= box * np.round(rij / box)                             # apply minimum image convention
    r = sqrt(np.sum(rij*rij,axis=-1))                            # r is now the NxN matrix of distances
    fill_diagonal(r,1.)                                          # set diagonal elements of the distance matrix to some arbitrary non-zero value to avoid dividing by zero when evaluating the force
    uij = uvec(r)
    np.fill_diagonal(uij,0.)                                     # zero out the diagonal elements of the potential energy matrix
    Epot = 0.5*np.sum(uij)

    # return the copmuted energy to the calling program
    return Epot

Now let's run the same simulation as before one more time. Because the code is so much faster now, we can run the simulation for more timesteps:

In [ ]:
configuration = loadtxt("N125_L7.469.dat")
box = configuration[0]
positions = configuration[1:]
velocities = zeros_like(positions)

Nsteps = 5000

grofilename = "output.gro"
grofile = open(grofilename,"w")          # open file to save trajectories

energyfilename = "energy.dat"
energyfile = open(energyfilename,"w")    # open file to save energies

forces = compute_forces()                # compute force for initial condition
for step in range(Nsteps):
    write_configuration(grofile)         # save configuration to file
    print (step*dt, compute_potential_energy(), compute_kinetic_energy(),file=energyfile)    # save energies to file
    perform_timestep()                   # perform Velocity Verlet timestep

energyfile.close()
grofile.close()

As always we should take a look at the energies to make sure that total energy is still conserved:

In [ ]:
energy = loadtxt(energyfilename)
times = energy[:,0]    # the first column
Epot = energy[:,1]     # the second column
Ekin = energy[:,2]     # the third column
plot (times, Epot, label="Potential Energy")
plot (times, Ekin, label="Kinetic Energy")
plot (times, Epot+Ekin, label="Total Energy")
xlabel("Time")
legend(loc="lower right")

## Controlling the temperature

We know from basic statistical mechanics that the average energy of a degree of freedom that appears quadratically in the Hamiltonian is $kT/2$. In our Lennard-Jones system of N atoms there are 3N momenta (the momentum of each atom in the x, y and z directions). However, not all of these 3N degrees of freedom are independent: the total momentum of the system is conserved in Newtonian dynamics, which imposes 3 constraints on the momenta. There are therefore $3N-3$ independent moementa in our system, which means that 
$$<E_{\text{kin}}> = (3N-3) kT / 2$$
or, equivalently,
$$ kT = 2 <E_{\text{kin}}> / (3N-3) $$.
Notice that this relationship expresses the *thermodynamic* temperature in terms of the *average* kinetic energy. For a give configuration we can define an *instantaneous* temperature
$$ kT_{\text{inst}} = 2 E_{\text{kin}} / (3N-3) $$
that would become the thermodynamic temperature when taking an ensemble average. Let's write this in code:

In [ ]:
def compute_instantaneous_temperature():
    """Computes the instantaneous temperature of the system."""

    Ekin = compute_kinetic_energy()
    N = positions.shape[0]
    return 2.0 * Ekin / (3*N - 3)

Let's look at the instantaneous temperature from our last simulation:

In [ ]:
plot (times, 2*Ekin/(3*125-3), label="Instantaneous temperature")
xlabel("Time")
legend(loc="lower right")

It looks like our system had a temperature of about 0.7 (in reduced units). So far we have no real control over this number; the total energy of the system (and therefore the average kinetic energy) were determined by the initial condition.

In practice we would like to *set* the temperature to a desired value, just as we do in experiments. One particularly simple way to achieve this is to change all atom velocities every so often by just the right amount so that the instantaneous temperature becomes the target temperature. If we do that often enough the system will eventually stay close to the target temperature.

From the above equation we can see that if the instantaneous temperature is $T_{\text{inst}}$ and we would like it to be equal to the target temperature $T$, we can simply multiply all velocities by $\sqrt{T / T_{\text{inst}}}$:

In [ ]:
def rescale_velocities(target_temperature):
    """Rescale velocities so that instantaneous temperature is equal to target temperature."""

    current_temperature = compute_instantaneous_temperature()
    if current_temperature > 0:
        scalefactor = np.sqrt(target_temperature / current_temperature)
        velocities[:] *= scalefactor
    return

Let's run the same simulation as before, but every 50 timesteps we rescale the velocities to reach a target temperature of 1.5 (in reduced units):

In [ ]:
configuration = loadtxt("N125_L7.469.dat")
box = configuration[0]
positions = configuration[1:]
velocities = zeros_like(positions)

Nsteps = 5000

grofilename = "output.gro"
grofile = open(grofilename,"w")          # open file to save trajectories

energyfilename = "energy.dat"
energyfile = open(energyfilename,"w")    # open file to save energies

forces = compute_forces()                # compute force for initial condition
for step in range(Nsteps):
    write_configuration(grofile)         # save configuration to file
    if step % 50 == 0:                   # every 50th step:
        rescale_velocities(1.5)          #   rescale velocities to target temperature
    print (step*dt, compute_potential_energy(), compute_kinetic_energy(),file=energyfile)    # save energies to file
    perform_timestep()                   # perform Velocity Verlet timestep

energyfile.close()
grofile.close()

Here is what the energies look like for this simulation...

In [ ]:
energy = loadtxt(energyfilename)
times = energy[:,0]    # the first column
Epot = energy[:,1]     # the second column
Ekin = energy[:,2]     # the third column
plot (times, Epot, label="Potential Energy")
plot (times, Ekin, label="Kinetic Energy")
plot (times, Epot+Ekin, label="Total Energy")
xlabel("Time")
legend(loc="lower right")

... and here is the instantaneous temperature:

In [ ]:
plot (times, 2*Ekin/(3*125-3), label="Instantaneous temperature")
xlabel("Time")
legend(loc="lower right")

#### Exercise 3: Watch the trajectory in VMD! What do you notice about the energies of this temperature-controlled simulation? Are you concerned?

#### Exercise 4: In VMD, go to Extension -> Analysis -> Radial Pair Distribution Function g(r). This allows you to compute the pair correlation function from your simulation data. Enter `all` for "Selection 1" and "Selection 2", and ignore the first 1000 frames when calculating the time average (why?). Calculate g(r) up to a maximum distance of 15 Angstrom. 